In [1]:
print("hello world")

hello world


In [2]:

import csv

val = []
test = []
neg, pos = 0, 0
with open("../dataset/original/aggre_fact_sota.csv", "r") as f:
    csvreader = list(csv.reader(f))
    csvreader = csvreader[1:]
    # print(len(csvreader))
    for row in csvreader:
        dataset = row[0]
        origin = row[1]
        cut = row[7]
        label = row[6]

        if dataset not in ["CLIFF", "Goyal21"]:
            continue
        if origin == "cnndm":
            continue


        if cut == "val":
            val.append(row)
        else:
            test.append(row)
            if label == "1":
                pos += 1
            else:
                neg += 1

print(len(val), len(test), neg, pos)     


200 200 111 89


In [3]:
def acc_distinguish_threshold(cuts: list, th: float) -> float:
    correct = 0
    for row in cuts:
        dae_score = float(row[8])
        label = int(row[6])
        if dae_score >= th:
            correct += 1 if label == 1 else 0
        else:
            correct += 1 if label == 0 else 0
    return correct / len(cuts)    

In [4]:
from sklearn.metrics import balanced_accuracy_score

def bacc_distinguish_threshold(cuts: list, th: float) -> float:
    y_true = [int(row[6]) for row in cuts]
    y_pred = [1 if float(row[8]) >= th else 0 for row in cuts]
    balanced_acc = balanced_accuracy_score(y_true, y_pred)
    return balanced_acc

In [5]:
def acc_decide_threshold(val: list) -> float:
    ths = [i / 1000 for i in range(1001)]
    best_th = 0
    max_score = 0
    for th in ths:
        score = acc_distinguish_threshold(val, th)
        # print(th, score)
        if score > max_score:
            max_score = score
            best_th = th
    return best_th

In [6]:
def bacc_decide_threshold(val: list) -> float:
    ths = [i / 1000 for i in range(1001)]
    best_th = 0
    max_score = 0
    for th in ths:
        score = bacc_distinguish_threshold(val, th)
        # print(th, score, max_score, best_th)
        if score > max_score:
            max_score = score
            best_th = th
    return best_th

In [7]:
def acc_calc_score(val: list, test: list) -> float:
    th = acc_decide_threshold(val)
    score = acc_distinguish_threshold(test, th)
    return score

In [8]:
def bacc_calc_score(val: list, test: list) -> float:
    th = bacc_decide_threshold(val)
    print(th)
    score = bacc_distinguish_threshold(test, th)
    return score

In [9]:
print(acc_calc_score(val, test), bacc_calc_score(val, test))

0.27
0.725 0.7155076424739346


In [10]:
from sklearn.metrics import confusion_matrix

th = acc_decide_threshold(val)
y_true = [int(row[6]) for row in test]
y_pred = [1 if float(row[8]) >= th else 0 for row in test]

cm = confusion_matrix(y_true, y_pred)

print(cm)

[[89 22]
 [33 56]]


In [11]:
from sklearn.metrics import confusion_matrix

th = bacc_decide_threshold(val)
y_true = [int(row[6]) for row in test]
y_pred = [1 if float(row[8]) >= th else 0 for row in test]

cm = confusion_matrix(y_true, y_pred)

print(cm)

[[89 22]
 [33 56]]


In [12]:
def classify_cm(val: list, test: list):
    fp, fn = [], []
    th = bacc_decide_threshold(val)
    for row in test:
        dae_score = float(row[8])
        label = int(row[6])
        pred = 1 if dae_score > th else 0
        row.append(pred)

        if label == 0 and pred == 1:
            fp.append(row)
        elif label == 1 and pred == 0:
            fn.append(row)
    return fp, fn
        

In [13]:
import random

fp, fn = classify_cm(val, test)
fpn = fp + fn
random.shuffle(fpn)
print(len(fpn), fpn[0])

55 ['CLIFF', 'xsum', '32446508', 'The 56ft (17.1m) converted trawler was 6 miles (10 km) west of South Stack when the crew radioed coastguards at 07:00 BST. Holyhead coastguards said a pump on board the boat was unable to cope with the water coming in. The RNLI lifeboat crew placed a pump and towed the vessel to Holyhead.', 'A fishing boat has been towed to safety after getting into difficulty in rough seas off the Anglesey coast.', 'BART', '1', 'test', '0.0037364955', '0.0', '0.318048640634074', '0.0', '0.0441055297851562', '1.0', '0.2116393744945526', '0.0', '1.8912305831909184', '1.0', 0]


In [14]:
print(str(fpn[0]))

['CLIFF', 'xsum', '32446508', 'The 56ft (17.1m) converted trawler was 6 miles (10 km) west of South Stack when the crew radioed coastguards at 07:00 BST. Holyhead coastguards said a pump on board the boat was unable to cope with the water coming in. The RNLI lifeboat crew placed a pump and towed the vessel to Holyhead.', 'A fishing boat has been towed to safety after getting into difficulty in rough seas off the Anglesey coast.', 'BART', '1', 'test', '0.0037364955', '0.0', '0.318048640634074', '0.0', '0.0441055297851562', '1.0', '0.2116393744945526', '0.0', '1.8912305831909184', '1.0', 0]


In [15]:
import json

def contains_others(lst, target):
    for item in lst:
        if item != target:
            return True
    return False

with open("../work/investigation.txt", "w") as f:
    for row in fpn:
        doc = row[3]
        sum = row[4]
        model = row[5]
        label = row[6]


        # bart
        input("document: ")
        f.write("document: {}\n".format(doc))
        print("doc: ", doc, "\n", flush=True)
        input("{}: ".format(model))
        f.write("sum: {}\n".format(sum))
        print("sum: ", sum, "\n", flush=True)
        pred = input("your_prediction: ")
        f.write("label: {}, your_pred: {}\n".format(label, pred))
        print("label: {}, your_pred: {}\n\n".format(label, pred), flush=True)
        f.write(str(row) + "\n\n")

doc:  The 56ft (17.1m) converted trawler was 6 miles (10 km) west of South Stack when the crew radioed coastguards at 07:00 BST. Holyhead coastguards said a pump on board the boat was unable to cope with the water coming in. The RNLI lifeboat crew placed a pump and towed the vessel to Holyhead. 

sum:  A fishing boat has been towed to safety after getting into difficulty in rough seas off the Anglesey coast. 

label: 1, your_pred: 0


doc:  no international side has toured bangladesh since 20 people were killed in a siege at a cafe in dhaka in july. the england and wales cricket board said in august that tour would go ahead following a security review. jos buttler will captain the one - day side, with the squads for the one - day and test series to be named on friday. the tour features three one - day internationals followed by two test matches. morgan has followed his own course, with the assurance ringing in his ears that nothing is going to happen, and i don't see how they can now s